<a href="https://colab.research.google.com/github/sidgyl-anz/multi-modal-research-agent/blob/main/LangGraph_Agent_API_Caller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calling the LangGraph Multi-Modal Research Agent API

This notebook demonstrates how to call the deployed LangGraph research agent API.

In [8]:
# Install necessary libraries
!pip install requests -q

In [ ]:
import requests
import json
import os

## Test Case 1: "Topic Only" Research

In [ ]:
import requests
import json

BASE_URL = "https://multi-modal-researcher-675059836631.us-central1.run.app"
GRAPH_ID = "research_agent"
HEADERS = {
    "Content-Type": "application/json",
}


def get_assistant_id_by_graph(graph_id):
    response = requests.post(
        f"{BASE_URL}/assistants/search",
        headers=HEADERS,
        json={"graph_id": graph_id, "limit": 1}
    )
    if response.status_code == 200:
        assistants = response.json()
        if assistants:
            return assistants[0].get("assistant_id")
    print("Failed to find assistant:", response.text)
    return None


def call_research_agent_with_assistant_api(assistant_id, input_payload):
    # Step 1: Create a Thread
    thread_response = requests.post(
        f"{BASE_URL}/threads",
        headers=HEADERS,
        json={"metadata": {"purpose": "research_test"}}
    )
    if thread_response.status_code != 200:
        print("Failed to create thread:", thread_response.text)
        return None

    thread_id = thread_response.json().get("thread_id")

    # Step 2: Run Assistant on Thread (wait for output)
    run_response = requests.post(
        f"{BASE_URL}/threads/{thread_id}/runs/wait",
        headers=HEADERS,
        json={
            "assistant_id": assistant_id,
            "input": input_payload["input"]
        }
    )
    if run_response.status_code != 200:
        print("Failed to run assistant:", run_response.text)
        return None

    return run_response.json()


# ---------- Workflow ----------
assistant_id = get_assistant_id_by_graph(GRAPH_ID)
if not assistant_id:
    raise RuntimeError("Assistant not found for graph_id 'research_agent'")

print("\n--- Test Case 1: Topic Only Research ---")
topic_only_input_payload = {
    "input": {
        "topic": "The history and evolution of functional programming languages",
        "research_approach": "Topic Only",
        "company_name": None,
        "title_areas": None,
        "video_url": None,
        "create_podcast": False
    }
}

results_topic_only = call_research_agent_with_assistant_api(assistant_id, topic_only_input_payload)

if results_topic_only:
    print("\n--- Full Response for Topic Only Research ---")
    print(json.dumps(results_topic_only, indent=2))
else:
    print("Failed to get results for Topic Only research.")


print("\n\n--- Test Case 2: Topic Company Leads Research ---")
topic_company_leads_input_payload = {
    "input": {
        "topic": "The role of Vector Databases in Generative AI applications",
        "research_approach": "Topic Company Leads",
        "company_name": "Pinecone",
        "title_areas": ["Solutions Architect", "Developer Advocate", "Head of Engineering"],
        "video_url": None,
        "create_podcast": False
    }
}

results_company_leads = call_research_agent_with_assistant_api(assistant_id, topic_company_leads_input_payload)

if results_company_leads:
    print("\n--- Full Response for Topic Company Leads Research ---")
    print(json.dumps(results_company_leads, indent=2))
else:
    print("Failed to get results for Topic Company Leads research.")




--- Test Case 1: Topic Only Research ---
